In [1]:
from pyspark.sql import SparkSession
from os.path import abspath

warehouse_location = abspath('spark-warehouse')

spark = SparkSession.builder    \
    .config('spark.driver.memory', '32g')   \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.sql.catalogImplementation","hive") \
    .enableHiveSupport()   \
    .getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/16 02:24:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [52]:
orders_sdf = spark.read.csv('data/instacart_market/orders.csv', header=True, inferSchema=True)
trains_sdf = spark.read.csv('data/instacart_market/order_products_train.csv', header=True, inferSchema=True)
priors_sdf = spark.read.csv('data/instacart_market/order_products_prior.csv', header=True, inferSchema=True)
products_sdf = spark.read.csv('data/instacart_market/products.csv', header=True, inferSchema=True)
aisles_sdf = spark.read.csv('data/instacart_market/aisles.csv', header=True, inferSchema=True)
depts_sdf = spark.read.csv('data/instacart_market/departments.csv', header=True, inferSchema=True)

In [53]:
sdf_dict = {
    'orders': orders_sdf,
    'trains': trains_sdf,
    'priors': priors_sdf,
    'products': products_sdf,
    'aisles': aisles_sdf,
    'depts': depts_sdf,
}
for sdf_name, sdf in sdf_dict.items():
    print(f'{sdf_name} schema')
    print(sdf.printSchema())

orders schema
root
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- eval_set: string (nullable = true)
 |-- order_number: integer (nullable = true)
 |-- order_dow: integer (nullable = true)
 |-- order_hour_of_day: integer (nullable = true)
 |-- days_since_prior_order: double (nullable = true)

None
trains schema
root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)

None
priors schema
root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)

None
products schema
root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)

None
aisles schema
root
 |-- aisle_id: integer (nullable = true)
 |

In [5]:
for name, sdf in sdf_dict.items():
    sdf.createOrReplaceTempView(name)

In [3]:
spark.catalog.listTables()

[Table(name='data_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order_priors_prods', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order_trains_prods', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='prd_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='test_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='train_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='up_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='user_mart', catalog='spark_catalog', namespace=

In [11]:
spark.sql('''
drop table if exists order_priors_prods;
''').show()

++
||
++
++



In [12]:
spark.sql('''
create table order_priors_prods 
as
select p.order_id, p.product_id, p.add_to_cart_order, p.reordered,
    o.user_id, o.eval_set, o.order_number, o.order_dow, o.order_hour_of_day,
    o.days_since_prior_order
from priors p, orders o
where p.order_id = o.order_id
''').show()

23/06/14 04:54:44 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/06/14 04:54:44 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/06/14 04:54:44 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/06/14 04:54:44 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/06/14 04:54:44 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/06/14 04:54:44 WARN HiveMetaStore: Location: file:/workspace/apache-spark-ml/spark-warehouse/order_priors_prods specified for non-external table:order_priors_prods


++
||
++
++



In [13]:
spark.catalog.listTables()

[Table(name='order_priors_prods', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='aisles', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='depts', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='orders', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='priors', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='products', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='trains', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [14]:
spark.sql('''
drop table if exists prd_mart
''').show()

++
||
++
++



In [19]:
query = '''
create table prd_mart
as
with
order_prods_grp as
(  
    select o.product_id
    , sum(case when reordered = 1 then 1 else 0 end) as prd_reordered_cnt
    , sum(case when reordered = 0 then 1 else 0 end) as prd_no_reordered_cnt
    , avg(reordered) as prd_avg_reordered
    , count(distinct user_id) prd_unq_usr_cnt
    , count(*) prd_total_cnt
    , count(distinct user_id) / count(*) as prd_usr_ratio
    , max(a.aisle_id) aisle_id
    , nvl(avg(days_since_prior_order), 0) as prd_avg_prior_days
    , nvl(min(days_since_prior_order), 0) as prd_min_prior_days
    , nvl(max(days_since_prior_order), 0) as prd_max_prior_days
    from order_priors_prods o, products p, aisles a
    where o.product_id = p.product_id
    and p.aisle_id = a.aisle_id
    group by o.product_id
),
order_aisles_grp as
(
    select a.aisle_id as aisle_id
    , count(distinct o.user_id) as aisle_distinct_usr_cnt
    , count(*) as aisle_total_cnt
    , count(distinct o.user_id) / count(*) as aisle_usr_ratio
    from order_priors_prods o, products p, aisles a
    where o.product_id = p.product_id
    and p.aisle_id = a.aisle_id
    group by a.aisle_id
),
order_prd_grp_aisle as
(
    select product_id, prd_reordered_cnt, prd_no_reordered_cnt, prd_avg_reordered, prd_unq_usr_cnt, prd_total_cnt, prd_usr_ratio
    , prd_avg_prior_days, prd_min_prior_days, prd_max_prior_days
    , aisle_distinct_usr_cnt, aisle_total_cnt, aisle_usr_ratio
    , prd_usr_ratio - aisle_usr_ratio as usr_ratio_diff
    from order_prods_grp op, order_aisles_grp oa
    where op.aisle_id = oa.aisle_id
)
select * from order_prd_grp_aisle
'''
spark.sql(query).show()

23/06/14 05:19:58 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/06/14 05:19:58 WARN HiveMetaStore: Location: file:/workspace/apache-spark-ml/spark-warehouse/prd_mart specified for non-external table:prd_mart


++
||
++
++



In [20]:
query = '''
select * from prd_mart limit 10
'''
spark.sql(query).show(truncate=False)

+----------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+-------------------+
|product_id|prd_reordered_cnt|prd_no_reordered_cnt|prd_avg_reordered  |prd_unq_usr_cnt|prd_total_cnt|prd_usr_ratio      |prd_avg_prior_days|prd_min_prior_days|prd_max_prior_days|aisle_distinct_usr_cnt|aisle_total_cnt|aisle_usr_ratio    |usr_ratio_diff     |
+----------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+-------------------+
|28884     |67               |66                  |0.5037593984962406 |66             |133          |0.49624060150375937|14.370689655172415|0.0               |30.0              |85357                 |575881         |0.1482198

In [21]:
query = '''
select count(*) from prd_mart
'''
spark.sql(query).show(truncate=False)

+--------+
|count(1)|
+--------+
|49676   |
+--------+



In [23]:
import pyspark.sql.functions as F

prd_mart_sdf = spark.sql('select * from prd_mart')
prd_mart_sdf.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in prd_mart_sdf.columns]).show()

+----------+-----------------+--------------------+-----------------+---------------+-------------+-------------+------------------+------------------+------------------+----------------------+---------------+---------------+--------------+
|product_id|prd_reordered_cnt|prd_no_reordered_cnt|prd_avg_reordered|prd_unq_usr_cnt|prd_total_cnt|prd_usr_ratio|prd_avg_prior_days|prd_min_prior_days|prd_max_prior_days|aisle_distinct_usr_cnt|aisle_total_cnt|aisle_usr_ratio|usr_ratio_diff|
+----------+-----------------+--------------------+-----------------+---------------+-------------+-------------+------------------+------------------+------------------+----------------------+---------------+---------------+--------------+
|         0|                0|                   0|                0|              0|            0|            0|                 0|                 0|                 0|                     0|              0|              0|             0|
+----------+-----------------+------

In [ ]:
spark.sql('''
drop table if exists user_mart_01
''').show()

In [25]:
query = '''
create table user_mart_01
as
select user_id
, count(*) as usr_total_cnt
, count(distinct product_id) prd_uq_cnt
, count(distinct order_id) order_uq_cnt
, count(*) / count(distinct product_id) usr_avg_uq_prd_cnt
, count(*) / count(distinct order_id) usr_avg_prd_cnt
, count(distinct product_id) / count(*) usr_uq_prd_ratio
, sum(reordered) usr_reord_cnt
, sum(case when reordered = 0 then 1 else 0 end) as usr_no_reord_cnt
, avg(reordered) usr_reordered_avg
, avg(days_since_prior_order) usr_avg_prior_days
, max(days_since_prior_order) usr_max_prior_days
, min(days_since_prior_order) usr_min_prior_days
, avg(order_dow) usr_avg_order_Dow
, avg(order_hour_of_day) usr_avg_order_hour_of_day
, max(order_number) as usr_max_order_number
from order_priors_prods o
group by user_id
'''
spark.sql(query).show()

23/06/14 06:41:53 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/06/14 06:41:53 WARN HiveMetaStore: Location: file:/workspace/apache-spark-ml/spark-warehouse/user_mart_01 specified for non-external table:user_mart_01
23/06/14 06:41:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


++
||
++
++



In [27]:
query = '''
select count(*) from user_mart_01
'''
spark.sql(query).show(truncate=False)

+--------+
|count(1)|
+--------+
|206209  |
+--------+



In [28]:
spark.sql('''
drop table if exists user_mart
''').show()

++
||
++
++



In [29]:
query = '''
create table user_mart
as
select u.*, o.order_id, o.eval_set, o.days_since_prior_order
from user_mart_01 u, orders o
where u.user_id = o.user_id
and o.eval_set in ('train', 'test')
'''
spark.sql(query).show()

23/06/14 06:48:49 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/06/14 06:48:49 WARN HiveMetaStore: Location: file:/workspace/apache-spark-ml/spark-warehouse/user_mart specified for non-external table:user_mart


++
||
++
++



In [30]:
query = '''
select * from user_mart limit 10
'''
spark.sql(query).show()

+-------+-------------+----------+------------+------------------+------------------+-------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+--------+--------+----------------------+
|user_id|usr_total_cnt|prd_uq_cnt|order_uq_cnt|usr_avg_uq_prd_cnt|   usr_avg_prd_cnt|   usr_uq_prd_ratio|usr_reord_cnt|usr_no_reord_cnt|  usr_reordered_avg|usr_avg_prior_days|usr_max_prior_days|usr_min_prior_days| usr_avg_order_Dow|usr_avg_order_hour_of_day|usr_max_order_number|order_id|eval_set|days_since_prior_order|
+-------+-------------+----------+------------+------------------+------------------+-------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+--------+--------+----------------------+
| 165817|          303|        85|   

In [31]:
query = '''
select count(*) from user_mart
'''
spark.sql(query).show(truncate=False)

+--------+
|count(1)|
+--------+
|206209  |
+--------+



In [33]:
query = '''
select count(*) from orders o
where o.eval_set in ('train', 'test')
'''
spark.sql(query).show(truncate=False)

+--------+
|count(1)|
+--------+
|206209  |
+--------+



In [34]:
query = '''
select * from orders where user_id = 1
'''
spark.sql(query).show(truncate=False)

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
|2539329 |1      |prior   |1           |2        |8                |null                  |
|2398795 |1      |prior   |2           |3        |7                |15.0                  |
|473747  |1      |prior   |3           |3        |12               |21.0                  |
|2254736 |1      |prior   |4           |4        |7                |29.0                  |
|431534  |1      |prior   |5           |4        |15               |28.0                  |
|3367565 |1      |prior   |6           |2        |7                |19.0                  |
|550135  |1      |prior   |7           |1        |9                |20.0                  |
|3108588 |1      |prior   |8           |1        |14               |14.0        

In [35]:
query = '''
select * from orders where user_id = 3
'''
spark.sql(query).show(truncate=False)

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
|1374495 |3      |prior   |1           |1        |14               |null                  |
|444309  |3      |prior   |2           |3        |19               |9.0                   |
|3002854 |3      |prior   |3           |3        |16               |21.0                  |
|2037211 |3      |prior   |4           |2        |18               |20.0                  |
|2710558 |3      |prior   |5           |0        |17               |12.0                  |
|1972919 |3      |prior   |6           |0        |16               |7.0                   |
|1839752 |3      |prior   |7           |0        |15               |7.0                   |
|3225766 |3      |prior   |8           |0        |17               |7.0         

In [ ]:
spark.sql('''
drop table if exists up_mart
''').show()

In [36]:
query = '''
select * from order_priors_prods
where user_id = 1
'''
spark.sql(query).show(truncate=False)

+--------+----------+-----------------+---------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|product_id|add_to_cart_order|reordered|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+----------+-----------------+---------+-------+--------+------------+---------+-----------------+----------------------+
|431534  |196       |1                |1        |1      |prior   |5           |4        |15               |28.0                  |
|431534  |12427     |2                |1        |1      |prior   |5           |4        |15               |28.0                  |
|431534  |10258     |3                |1        |1      |prior   |5           |4        |15               |28.0                  |
|431534  |25133     |4                |1        |1      |prior   |5           |4        |15               |28.0                  |
|431534  |10326     |5                |0        |1      |prior   |5           |4   

In [38]:
query = '''
create table up_mart
as
with
up_grp as 
(
    select user_id, product_id
    , count(*) up_cnt
    , sum(reordered) up_reord_cnt
    , sum(case when reordered = 0 then 1 else 0 end) up_no_reord_cnt
    , avg(reordered) up_reordered_avg
    , max(order_number) up_max_ord_num
    , min(order_number) up_min_ord_num
    , avg(add_to_cart_order) up_avg_cart
    , avg(days_since_prior_order) as up_avg_prior_days
    , max(days_since_prior_order) as up_max_prior_days
    , min(days_since_prior_order) as up_min_prior_days
    , avg(order_dow) as up_avg_ord_dow
    , avg(order_hour_of_day) as up_avg_ord_hour
    from order_priors_prods
    group by user_id, product_id
)
select up.*
, up.up_cnt / um.usr_total_cnt as up_usr_ratio
, up.up_reord_cnt / um.usr_reord_cnt as up_usr_reord_ratio
, um.usr_reord_cnt
, um.usr_max_order_number - up.up_max_ord_num as up_usr_ord_num_diff
from up_grp up, user_mart um
where up.user_id = um.user_id
'''
spark.sql(query).show(truncate=False)

23/06/14 07:38:24 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/06/14 07:38:24 WARN HiveMetaStore: Location: file:/workspace/apache-spark-ml/spark-warehouse/up_mart specified for non-external table:up_mart


++
||
++
++



In [43]:
query = '''
select count(*) from up_mart
'''
spark.sql(query).show(truncate=False)

+--------+
|count(1)|
+--------+
|13307953|
+--------+



In [39]:
query = '''
select * from user_mart where usr_reord_cnt is null
'''
spark.sql(query).show(truncate=False)

+-------+-------------+----------+------------+------------------+---------------+----------------+-------------+----------------+-----------------+------------------+------------------+------------------+-----------------+-------------------------+--------------------+--------+--------+----------------------+
|user_id|usr_total_cnt|prd_uq_cnt|order_uq_cnt|usr_avg_uq_prd_cnt|usr_avg_prd_cnt|usr_uq_prd_ratio|usr_reord_cnt|usr_no_reord_cnt|usr_reordered_avg|usr_avg_prior_days|usr_max_prior_days|usr_min_prior_days|usr_avg_order_Dow|usr_avg_order_hour_of_day|usr_max_order_number|order_id|eval_set|days_since_prior_order|
+-------+-------------+----------+------------+------------------+---------------+----------------+-------------+----------------+-----------------+------------------+------------------+------------------+-----------------+-------------------------+--------------------+--------+--------+----------------------+
+-------+-------------+----------+------------+-----------------

In [40]:
query = '''
select * from up_mart where usr_reord_cnt is null
'''
spark.sql(query).show(truncate=False)

+-------+----------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+------------+------------------+-------------+-------------------+
|user_id|product_id|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days|up_avg_ord_dow|up_avg_ord_hour|up_usr_ratio|up_usr_reord_ratio|usr_reord_cnt|up_usr_ord_num_diff|
+-------+----------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+------------+------------------+-------------+-------------------+
+-------+----------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------

In [41]:
query = '''
select * from up_mart where up_usr_reord_ratio is null
'''
spark.sql(query).show(truncate=False)

+-------+----------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+------------------+-------------+-------------------+
|user_id|product_id|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days|up_avg_ord_dow|up_avg_ord_hour|up_usr_ratio        |up_usr_reord_ratio|usr_reord_cnt|up_usr_ord_num_diff|
+-------+----------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+------------------+-------------+-------------------+
|40232  |39561     |1     |0           |1              |0.0             |2             |2             |10.0       |13.0             |13.0             |13.0             |2.0   

In [60]:
query = '''
select count(*)
from up_mart up
left outer join user_mart u
on up.user_id = u.user_id
left outer join prd_mart p
on up.product_id = p.product_id
where (u.user_id is null or p.product_id is null)
'''
spark.sql(query).show(truncate=False)

+--------+
|count(1)|
+--------+
|3       |
+--------+



In [46]:
query = '''
select 'user_mart_count' as class, count(*) from user_mart
union all
select 'prd_mart_count' as class, count(*) from prd_mart
union all
select 'up_mart_count' as class, count(*) from up_mart
'''
spark.sql(query).show(truncate=False)

+---------------+--------+
|class          |count(1)|
+---------------+--------+
|user_mart_count|206209  |
|prd_mart_count |49676   |
|up_mart_count  |13307953|
+---------------+--------+



In [49]:
query = '''
describe up_mart
'''
spark.sql(query).show(truncate=False)

+-------------------+---------+-------+
|col_name           |data_type|comment|
+-------------------+---------+-------+
|user_id            |int      |null   |
|product_id         |int      |null   |
|up_cnt             |bigint   |null   |
|up_reord_cnt       |bigint   |null   |
|up_no_reord_cnt    |bigint   |null   |
|up_reordered_avg   |double   |null   |
|up_max_ord_num     |int      |null   |
|up_min_ord_num     |int      |null   |
|up_avg_cart        |double   |null   |
|up_avg_prior_days  |double   |null   |
|up_max_prior_days  |double   |null   |
|up_min_prior_days  |double   |null   |
|up_avg_ord_dow     |double   |null   |
|up_avg_ord_hour    |double   |null   |
|up_usr_ratio       |double   |null   |
|up_usr_reord_ratio |double   |null   |
|usr_reord_cnt      |bigint   |null   |
|up_usr_ord_num_diff|int      |null   |
+-------------------+---------+-------+



In [52]:
print(spark.sql('select * from up_mart').columns)
print(spark.sql('select * from user_mart').columns)
print(spark.sql('select * from prd_mart').columns)

['user_id', 'product_id', 'up_cnt', 'up_reord_cnt', 'up_no_reord_cnt', 'up_reordered_avg', 'up_max_ord_num', 'up_min_ord_num', 'up_avg_cart', 'up_avg_prior_days', 'up_max_prior_days', 'up_min_prior_days', 'up_avg_ord_dow', 'up_avg_ord_hour', 'up_usr_ratio', 'up_usr_reord_ratio', 'usr_reord_cnt', 'up_usr_ord_num_diff']
['user_id', 'usr_total_cnt', 'prd_uq_cnt', 'order_uq_cnt', 'usr_avg_uq_prd_cnt', 'usr_avg_prd_cnt', 'usr_uq_prd_ratio', 'usr_reord_cnt', 'usr_no_reord_cnt', 'usr_reordered_avg', 'usr_avg_prior_days', 'usr_max_prior_days', 'usr_min_prior_days', 'usr_avg_order_Dow', 'usr_avg_order_hour_of_day', 'usr_max_order_number', 'order_id', 'eval_set', 'days_since_prior_order']
['product_id', 'prd_reordered_cnt', 'prd_no_reordered_cnt', 'prd_avg_reordered', 'prd_unq_usr_cnt', 'prd_total_cnt', 'prd_usr_ratio', 'prd_avg_prior_days', 'prd_min_prior_days', 'prd_max_prior_days', 'aisle_distinct_usr_cnt', 'aisle_total_cnt', 'aisle_usr_ratio', 'usr_ratio_diff']


In [ ]:
spark.sql('drop table if exists data_mart').show()

In [55]:
query = '''
create table data_mart
as
select up.user_id, up.product_id, u.order_id
, up_cnt, up_reord_cnt, up_no_reord_cnt, up_reordered_avg, up_max_ord_num, up_min_ord_num, up_avg_cart, up_avg_prior_days, up_max_prior_days
, up_min_prior_days, up_avg_ord_dow, up_avg_ord_hour, up_usr_ratio, up_usr_reord_ratio, up_usr_ord_num_diff

, usr_total_cnt, prd_uq_cnt, order_uq_cnt, usr_avg_prd_cnt, usr_avg_uq_prd_cnt, usr_uq_prd_ratio, u.usr_reord_cnt, usr_no_reord_cnt, usr_reordered_avg, usr_avg_prior_days
, usr_max_prior_days, usr_min_prior_days, usr_avg_order_dow, usr_avg_order_hour_of_day, usr_max_order_number, eval_set, days_since_prior_order

, prd_reordered_cnt, prd_no_reordered_cnt, prd_avg_reordered, prd_unq_usr_cnt, prd_total_cnt, prd_usr_ratio, prd_avg_prior_days, prd_min_prior_days, prd_max_prior_days
, aisle_distinct_usr_cnt, aisle_total_cnt, aisle_usr_ratio, usr_ratio_diff
from up_mart up, user_mart u, prd_mart p
where up.user_id = u.user_id
and up.product_id = p.product_id
'''
spark.sql(query).show()

23/06/14 10:12:05 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/06/14 10:12:05 WARN HiveMetaStore: Location: file:/workspace/apache-spark-ml/spark-warehouse/data_mart specified for non-external table:data_mart


++
||
++
++



In [57]:
query = '''
select 'data_mart count' as class, count(*) from data_mart
union all 
select 'user_mart count' as class, count(*) from user_mart
union all
select 'prd_mart count' as class, count(*) from prd_mart
union all
select 'up_mart count' as class, count(*) from up_mart
'''
spark.sql(query).show(truncate=False)

+---------------+--------+
|class          |count(1)|
+---------------+--------+
|data_mart count|13307950|
|user_mart count|206209  |
|prd_mart count |49676   |
|up_mart count  |13307953|
+---------------+--------+



In [58]:
spark.sql('select * from data_mart limit 10').show()

+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+------------------+-------------------+-------------+----------+------------+-----------------+------------------+------------------+-------------+----------------+------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+--------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days|up_avg_

In [61]:
spark.sql('select * from trains limit 10').show(truncate=False)

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|1       |49302     |1                |1        |
|1       |11109     |2                |1        |
|1       |10246     |3                |0        |
|1       |49683     |4                |0        |
|1       |43633     |5                |1        |
|1       |13176     |6                |0        |
|1       |47209     |7                |0        |
|1       |22035     |8                |1        |
|36      |39612     |1                |0        |
|36      |19660     |2                |1        |
+--------+----------+-----------------+---------+



In [62]:
spark.sql('select count(*) from trains').show(truncate=False)

+--------+
|count(1)|
+--------+
|1384617 |
+--------+



In [ ]:
spark.sql('drop table if exists order_trains_prods').show(truncate=False)

In [63]:
query = '''
create table order_trains_prods
as
select t.order_id, t.product_id, t.reordered, o.user_id
from trains t, orders o
where t.order_id = o.order_id
'''
spark.sql(query).show(truncate=False)

23/06/14 10:33:10 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/06/14 10:33:10 WARN HiveMetaStore: Location: file:/workspace/apache-spark-ml/spark-warehouse/order_trains_prods specified for non-external table:order_trains_prods


++
||
++
++



In [64]:
spark.sql('select count(*) from order_trains_prods').show()

+--------+
|count(1)|
+--------+
| 1384617|
+--------+



In [65]:
spark.sql('select * from order_trains_prods limit 10').show()

+--------+----------+---------+-------+
|order_id|product_id|reordered|user_id|
+--------+----------+---------+-------+
|    1001|     34966|        1|  47475|
|    1001|     23517|        1|  47475|
|    1001|     26369|        1|  47475|
|    1001|     43961|        1|  47475|
|    1620|     44626|        0| 157320|
|    1620|     15887|        0| 157320|
|    1620|      1269|        0| 157320|
|    1620|     11301|        1| 157320|
|    1620|     25743|        1| 157320|
|    1620|     34126|        1| 157320|
+--------+----------+---------+-------+



In [67]:
query = '''
select user_id, product_id, count(*)
from order_trains_prods
group by user_id, product_id
having count(*) > 1
'''
spark.sql(query).show()

+-------+----------+--------+
|user_id|product_id|count(1)|
+-------+----------+--------+
+-------+----------+--------+



In [68]:
query = '''
select count(*)
from order_trains_prods o
left outer join data_mart d
on o.user_id = d.user_id
and o.product_id = d.product_id
where d.product_id is null
'''
spark.sql(query).show(truncate=False)

+--------+
|count(1)|
+--------+
|555793  |
+--------+



In [69]:
query = '''
with
data_user_grp as 
(
    select user_id from data_mart group by user_id
),
data_product_grp as
(
    select product_id from data_mart group by product_id
)
select 'user_id_count' as class, count(*) from order_trains_prods o
left outer join data_user_grp d on o.user_id = d.user_id
where d.user_id is null
union all
select 'product_id_count' as class, count(*) from order_trains_prods o
left outer join data_product_grp d on o.product_id = d.product_id
where d.product_id is null
'''
spark.sql(query).show(truncate=False)

+----------------+--------+
|class           |count(1)|
+----------------+--------+
|user_id_count   |0       |
|product_id_count|9       |
+----------------+--------+



In [81]:
query = '''
select a.reordered, count(*) 
from order_trains_prods a
left outer join data_mart b
on a.user_id = b.user_id 
and a.product_id = b.product_id
where b.product_id is null
group by a.reordered
'''
spark.sql(query).show(truncate=False)

+---------+--------+
|reordered|count(1)|
+---------+--------+
|0        |555793  |
+---------+--------+



In [ ]:
spark.sql('drop table if exists train_data')

In [83]:
print(spark.sql('select * from data_mart').columns)

['user_id', 'product_id', 'order_id', 'up_cnt', 'up_reord_cnt', 'up_no_reord_cnt', 'up_reordered_avg', 'up_max_ord_num', 'up_min_ord_num', 'up_avg_cart', 'up_avg_prior_days', 'up_max_prior_days', 'up_min_prior_days', 'up_avg_ord_dow', 'up_avg_ord_hour', 'up_usr_ratio', 'up_usr_reord_ratio', 'up_usr_ord_num_diff', 'usr_total_cnt', 'prd_uq_cnt', 'order_uq_cnt', 'usr_avg_prd_cnt', 'usr_avg_uq_prd_cnt', 'usr_uq_prd_ratio', 'usr_reord_cnt', 'usr_no_reord_cnt', 'usr_reordered_avg', 'usr_avg_prior_days', 'usr_max_prior_days', 'usr_min_prior_days', 'usr_avg_order_dow', 'usr_avg_order_hour_of_day', 'usr_max_order_number', 'eval_set', 'days_since_prior_order', 'prd_reordered_cnt', 'prd_no_reordered_cnt', 'prd_avg_reordered', 'prd_unq_usr_cnt', 'prd_total_cnt', 'prd_usr_ratio', 'prd_avg_prior_days', 'prd_min_prior_days', 'prd_max_prior_days', 'aisle_distinct_usr_cnt', 'aisle_total_cnt', 'aisle_usr_ratio', 'usr_ratio_diff']


In [85]:
query = '''
create table train_data
as
select 
up_cnt, up_reord_cnt, up_no_reord_cnt, up_reordered_avg, up_max_ord_num, up_min_ord_num, up_avg_cart, up_avg_prior_days, up_max_prior_days, up_min_prior_days
, up_avg_ord_dow, up_avg_ord_hour, up_usr_ratio, up_usr_reord_ratio, up_usr_ord_num_diff, usr_total_cnt, prd_uq_cnt, order_uq_cnt, usr_avg_prd_cnt, usr_avg_uq_prd_cnt
, usr_uq_prd_ratio, usr_reord_cnt, usr_no_reord_cnt, usr_reordered_avg, usr_avg_prior_days, usr_max_prior_days, usr_min_prior_days, usr_avg_order_dow
, usr_avg_order_hour_of_day, usr_max_order_number
, days_since_prior_order, prd_reordered_cnt, prd_no_reordered_cnt, prd_avg_reordered, prd_unq_usr_cnt
, prd_total_cnt, prd_usr_ratio, prd_avg_prior_days, prd_min_prior_days, prd_max_prior_days, aisle_distinct_usr_cnt
, aisle_total_cnt, aisle_usr_ratio, usr_ratio_diff
, b.reordered -- nvl(b.reordered, 0)
from data_mart a left outer join order_trains_prods b
on a.user_id = b.user_id and a.product_id = b.product_id
where a.eval_set = 'train'
'''
spark.sql(query).show()

23/06/14 11:17:44 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/06/14 11:17:44 WARN HiveMetaStore: Location: file:/workspace/apache-spark-ml/spark-warehouse/train_data specified for non-external table:train_data


++
||
++
++



In [89]:
query = '''
select 'train_data' as class, count(*) from train_data
union all
select 'reordered null' as class, count(*) from train_data where reordered is null
'''
spark.sql(query).show()

+--------------+--------+
|         class|count(1)|
+--------------+--------+
|    train_data| 8474661|
|reordered null| 7645837|
+--------------+--------+



In [ ]:
spark.sql('drop table if exists test_data')

In [93]:
query = '''
create table test_data
as
select user_id, product_id, order_id
, up_cnt, up_reord_cnt, up_no_reord_cnt, up_reordered_avg, up_max_ord_num, up_min_ord_num, up_avg_cart, up_avg_prior_days, up_max_prior_days, up_min_prior_days
, up_avg_ord_dow, up_avg_ord_hour, up_usr_ratio, up_usr_reord_ratio, up_usr_ord_num_diff, usr_total_cnt, prd_uq_cnt, order_uq_cnt, usr_avg_prd_cnt, usr_avg_uq_prd_cnt
, usr_uq_prd_ratio, usr_reord_cnt, usr_no_reord_cnt, usr_reordered_avg, usr_avg_prior_days, usr_max_prior_days, usr_min_prior_days, usr_avg_order_dow
, usr_avg_order_hour_of_day, usr_max_order_number
, days_since_prior_order, prd_reordered_cnt, prd_no_reordered_cnt, prd_avg_reordered, prd_unq_usr_cnt
, prd_total_cnt, prd_usr_ratio, prd_avg_prior_days, prd_min_prior_days, prd_max_prior_days, aisle_distinct_usr_cnt
, aisle_total_cnt, aisle_usr_ratio, usr_ratio_diff
from data_mart a
where a.eval_set = 'test'
'''
spark.sql(query).show()

23/06/14 11:30:34 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/06/14 11:30:34 WARN HiveMetaStore: Location: file:/workspace/apache-spark-ml/spark-warehouse/test_data specified for non-external table:test_data


++
||
++
++



In [94]:
spark.sql('select count(*) from test_data').show(truncate=False)

+--------+
|count(1)|
+--------+
|4833289 |
+--------+



In [3]:
spark.catalog.listTables()

23/06/16 00:40:53 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/06/16 00:40:53 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/06/16 00:40:56 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/06/16 00:40:56 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.17.0.2
23/06/16 00:40:56 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


[Table(name='data_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order_priors_prods', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order_trains_prods', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='prd_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='test_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='train_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='up_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='user_mart', catalog='spark_catalog', namespace=

In [4]:
train_sdf = spark.sql('select * from train_data')
test_sdf = spark.sql('select * from test_data')
type(train_sdf), type(test_sdf)

(pyspark.sql.dataframe.DataFrame, pyspark.sql.dataframe.DataFrame)

In [5]:
train_sdf.printSchema()

root
 |-- up_cnt: long (nullable = true)
 |-- up_reord_cnt: long (nullable = true)
 |-- up_no_reord_cnt: long (nullable = true)
 |-- up_reordered_avg: double (nullable = true)
 |-- up_max_ord_num: integer (nullable = true)
 |-- up_min_ord_num: integer (nullable = true)
 |-- up_avg_cart: double (nullable = true)
 |-- up_avg_prior_days: double (nullable = true)
 |-- up_max_prior_days: double (nullable = true)
 |-- up_min_prior_days: double (nullable = true)
 |-- up_avg_ord_dow: double (nullable = true)
 |-- up_avg_ord_hour: double (nullable = true)
 |-- up_usr_ratio: double (nullable = true)
 |-- up_usr_reord_ratio: double (nullable = true)
 |-- up_usr_ord_num_diff: integer (nullable = true)
 |-- usr_total_cnt: long (nullable = true)
 |-- prd_uq_cnt: long (nullable = true)
 |-- order_uq_cnt: long (nullable = true)
 |-- usr_avg_prd_cnt: double (nullable = true)
 |-- usr_avg_uq_prd_cnt: double (nullable = true)
 |-- usr_uq_prd_ratio: double (nullable = true)
 |-- usr_reord_cnt: long (nulla

In [6]:
test_sdf.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- up_cnt: long (nullable = true)
 |-- up_reord_cnt: long (nullable = true)
 |-- up_no_reord_cnt: long (nullable = true)
 |-- up_reordered_avg: double (nullable = true)
 |-- up_max_ord_num: integer (nullable = true)
 |-- up_min_ord_num: integer (nullable = true)
 |-- up_avg_cart: double (nullable = true)
 |-- up_avg_prior_days: double (nullable = true)
 |-- up_max_prior_days: double (nullable = true)
 |-- up_min_prior_days: double (nullable = true)
 |-- up_avg_ord_dow: double (nullable = true)
 |-- up_avg_ord_hour: double (nullable = true)
 |-- up_usr_ratio: double (nullable = true)
 |-- up_usr_reord_ratio: double (nullable = true)
 |-- up_usr_ord_num_diff: integer (nullable = true)
 |-- usr_total_cnt: long (nullable = true)
 |-- prd_uq_cnt: long (nullable = true)
 |-- order_uq_cnt: long (nullable = true)
 |-- usr_avg_prd_cnt: double (nullable = true)
 |-- 

In [7]:
import pyspark.sql.functions as F

train_sdf.select([F.count(F.when(F.col(c).isNull() | F.isnan(F.col(c)), c)).alias(c) for c in train_sdf.columns]).show()

23/06/16 02:26:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/06/16 02:26:25 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+------------+------------------+-------------------+-------------+----------+------------+---------------+------------------+----------------+-------------+----------------+-----------------+------------------+------------------+------------------+-----------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-----------------+---------------+-------------+-------------+------------------+------------------+------------------+----------------------+---------------+---------------+--------------+---------+
|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days|up_avg_ord_dow|up_avg_ord_hour|up_usr_ratio|up_usr_reord_ratio|up_usr_ord_num_diff|usr_total_c

In [8]:
train_sdf = train_sdf.fillna(0)

In [9]:
train_sdf.select([F.count(F.when(F.col(c).isNull() | F.isnan(F.col(c)), c)).alias(c) for c in train_sdf.columns]).show()

+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+------------+------------------+-------------------+-------------+----------+------------+---------------+------------------+----------------+-------------+----------------+-----------------+------------------+------------------+------------------+-----------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-----------------+---------------+-------------+-------------+------------------+------------------+------------------+----------------------+---------------+---------------+--------------+---------+
|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days|up_avg_ord_dow|up_avg_ord_hour|up_usr_ratio|up_usr_reord_ratio|up_usr_ord_num_diff|usr_total_c

In [10]:
train_sdf.dtypes

[('up_cnt', 'bigint'),
 ('up_reord_cnt', 'bigint'),
 ('up_no_reord_cnt', 'bigint'),
 ('up_reordered_avg', 'double'),
 ('up_max_ord_num', 'int'),
 ('up_min_ord_num', 'int'),
 ('up_avg_cart', 'double'),
 ('up_avg_prior_days', 'double'),
 ('up_max_prior_days', 'double'),
 ('up_min_prior_days', 'double'),
 ('up_avg_ord_dow', 'double'),
 ('up_avg_ord_hour', 'double'),
 ('up_usr_ratio', 'double'),
 ('up_usr_reord_ratio', 'double'),
 ('up_usr_ord_num_diff', 'int'),
 ('usr_total_cnt', 'bigint'),
 ('prd_uq_cnt', 'bigint'),
 ('order_uq_cnt', 'bigint'),
 ('usr_avg_prd_cnt', 'double'),
 ('usr_avg_uq_prd_cnt', 'double'),
 ('usr_uq_prd_ratio', 'double'),
 ('usr_reord_cnt', 'bigint'),
 ('usr_no_reord_cnt', 'bigint'),
 ('usr_reordered_avg', 'double'),
 ('usr_avg_prior_days', 'double'),
 ('usr_max_prior_days', 'double'),
 ('usr_min_prior_days', 'double'),
 ('usr_avg_order_dow', 'double'),
 ('usr_avg_order_hour_of_day', 'double'),
 ('usr_max_order_number', 'int'),
 ('days_since_prior_order', 'double'),


In [11]:
vector_columns = [column for column in train_sdf.columns if column != 'reordered']
print(vector_columns)

['up_cnt', 'up_reord_cnt', 'up_no_reord_cnt', 'up_reordered_avg', 'up_max_ord_num', 'up_min_ord_num', 'up_avg_cart', 'up_avg_prior_days', 'up_max_prior_days', 'up_min_prior_days', 'up_avg_ord_dow', 'up_avg_ord_hour', 'up_usr_ratio', 'up_usr_reord_ratio', 'up_usr_ord_num_diff', 'usr_total_cnt', 'prd_uq_cnt', 'order_uq_cnt', 'usr_avg_prd_cnt', 'usr_avg_uq_prd_cnt', 'usr_uq_prd_ratio', 'usr_reord_cnt', 'usr_no_reord_cnt', 'usr_reordered_avg', 'usr_avg_prior_days', 'usr_max_prior_days', 'usr_min_prior_days', 'usr_avg_order_dow', 'usr_avg_order_hour_of_day', 'usr_max_order_number', 'days_since_prior_order', 'prd_reordered_cnt', 'prd_no_reordered_cnt', 'prd_avg_reordered', 'prd_unq_usr_cnt', 'prd_total_cnt', 'prd_usr_ratio', 'prd_avg_prior_days', 'prd_min_prior_days', 'prd_max_prior_days', 'aisle_distinct_usr_cnt', 'aisle_total_cnt', 'aisle_usr_ratio', 'usr_ratio_diff']


In [12]:
from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler(inputCols=vector_columns, outputCol='features')
train_sdf_vectorized = vector_assembler.transform(train_sdf)
train_sdf_vectorized.show(10)

+------+------------+---------------+------------------+--------------+--------------+------------------+------------------+-----------------+-----------------+------------------+------------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+---------+--------------------+
|up_cnt|up_reord_cnt|up_no_reord_cnt|  up_reordered_avg|up_max_ord_num|up_min_ord_num|       up_avg_cart| up_avg_prior_days|up_max_prior_days|up_min_prior_days|    up_avg

In [13]:
from pyspark.ml.classification import RandomForestClassifier

rf_estimator = RandomForestClassifier(labelCol='reordered')
rf_model = rf_estimator.fit(train_sdf_vectorized)

In [14]:
type(rf_model)

pyspark.ml.classification.RandomForestClassificationModel

In [15]:
test_sdf = spark.sql('select * from test_data')

23/06/16 02:27:47 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/06/16 02:27:47 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/06/16 02:27:47 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [16]:
test_sdf.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- up_cnt: long (nullable = true)
 |-- up_reord_cnt: long (nullable = true)
 |-- up_no_reord_cnt: long (nullable = true)
 |-- up_reordered_avg: double (nullable = true)
 |-- up_max_ord_num: integer (nullable = true)
 |-- up_min_ord_num: integer (nullable = true)
 |-- up_avg_cart: double (nullable = true)
 |-- up_avg_prior_days: double (nullable = true)
 |-- up_max_prior_days: double (nullable = true)
 |-- up_min_prior_days: double (nullable = true)
 |-- up_avg_ord_dow: double (nullable = true)
 |-- up_avg_ord_hour: double (nullable = true)
 |-- up_usr_ratio: double (nullable = true)
 |-- up_usr_reord_ratio: double (nullable = true)
 |-- up_usr_ord_num_diff: integer (nullable = true)
 |-- usr_total_cnt: long (nullable = true)
 |-- prd_uq_cnt: long (nullable = true)
 |-- order_uq_cnt: long (nullable = true)
 |-- usr_avg_prd_cnt: double (nullable = true)
 |-- 

In [17]:
test_id_columns = ['user_id', 'product_id', 'order_id']
test_sdf_id = test_sdf.select(*test_id_columns)
test_sdf = test_sdf.drop(*test_id_columns)

In [18]:
test_sdf.limit(10).show()

+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+-------------------+-------------------+-------------+----------+------------+---------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+-------------------+
|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days|up_avg_ord_dow|up_avg_ord_hour|        up_usr_ratio| up_usr_reord_ratio|u

In [19]:
test_sdf = test_sdf.fillna(0)
test_sdf_vectorized = vector_assembler.transform(test_sdf)

In [20]:
predictions = rf_model.transform(test_sdf_vectorized)
predictions.show(truncate=False)

+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+-------------------+-------------------+-------------+----------+------------+---------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+---------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
predictions.groupBy('prediction').count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|4833233|
|       1.0|     56|
+----------+-------+



In [22]:
test_sdf_id.limit(10).show()

+-------+----------+--------+
|user_id|product_id|order_id|
+-------+----------+--------+
|    141|      9387|  726111|
|    141|     47766|  726111|
|    141|     10941|  726111|
|    141|     32256|  726111|
|    141|     44632|  726111|
|    141|     21137|  726111|
|    141|     31717|  726111|
|    141|     17968|  726111|
|    141|     20734|  726111|
|    141|     34448|  726111|
+-------+----------+--------+



In [23]:
test_sdf_id.show()

+-------+----------+--------+
|user_id|product_id|order_id|
+-------+----------+--------+
|    141|      9387|  726111|
|    141|     47766|  726111|
|    141|     10941|  726111|
|    141|     32256|  726111|
|    141|     44632|  726111|
|    141|     21137|  726111|
|    141|     31717|  726111|
|    141|     17968|  726111|
|    141|     20734|  726111|
|    141|     34448|  726111|
|    141|      7388|  726111|
|    141|     31605|  726111|
|    141|     25666|  726111|
|    141|      9554|  726111|
|    141|     10761|  726111|
|    141|     14057|  726111|
|    141|     45603|  726111|
|    141|     41220|  726111|
|    141|     27845|  726111|
|    141|     17454|  726111|
+-------+----------+--------+
only showing top 20 rows



In [24]:
test_sdf_id = test_sdf_id.withColumn('row_id', F.monotonically_increasing_id())
predictions = predictions.withColumn('row_id', F.monotonically_increasing_id())
predictions.limit(10).show()

+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+-------------------+-------------------+-------------+----------+------------+---------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+-------------------+--------------------+--------------------+--------------------+----------+------+
|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_pri

In [25]:
predictions = test_sdf_id.join(predictions, 'row_id').drop('row_id')
print(test_sdf.count(), predictions.count())
predictions.limit(10).show()

4833289 4833289


+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_a

In [26]:
predictions.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- up_cnt: long (nullable = true)
 |-- up_reord_cnt: long (nullable = true)
 |-- up_no_reord_cnt: long (nullable = true)
 |-- up_reordered_avg: double (nullable = false)
 |-- up_max_ord_num: integer (nullable = true)
 |-- up_min_ord_num: integer (nullable = true)
 |-- up_avg_cart: double (nullable = false)
 |-- up_avg_prior_days: double (nullable = false)
 |-- up_max_prior_days: double (nullable = false)
 |-- up_min_prior_days: double (nullable = false)
 |-- up_avg_ord_dow: double (nullable = false)
 |-- up_avg_ord_hour: double (nullable = false)
 |-- up_usr_ratio: double (nullable = false)
 |-- up_usr_reord_ratio: double (nullable = false)
 |-- up_usr_ord_num_diff: integer (nullable = true)
 |-- usr_total_cnt: long (nullable = true)
 |-- prd_uq_cnt: long (nullable = true)
 |-- order_uq_cnt: long (nullable = true)
 |-- usr_avg_prd_cnt: double (nullable = fa

In [27]:
from pyspark.ml.functions import vector_to_array

predictions = predictions.withColumn('probability_arr', vector_to_array('probability'))
predictions.limit(10).show()

+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_nu

In [28]:
predictions.select(F.col('probability_arr')[1]).show(10)

+-------------------+
| probability_arr[1]|
+-------------------+
|0.08598726977746626|
|0.07651389341422152|
| 0.1277860354084583|
|0.23114361923631574|
| 0.1262760096516318|
|0.07570028595892689|
|0.07685726408204009|
|0.08891507873367385|
| 0.0874186619814454|
|0.06511441140735877|
+-------------------+
only showing top 10 rows



In [29]:
predictions = predictions.withColumn('1_proba', F.col('probability_arr')[1])
predictions.show(10)

+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------------------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reorder

In [30]:
REORDERED_THRESHOLD = 0.21

predictions = predictions.withColumn('reordered', (F.col('1_proba') > REORDERED_THRESHOLD).cast('int'))
predictions.show(10)

+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------------------+---------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|

In [31]:
predictions_reordered = predictions.filter(F.col('reordered') == 1)
predictions_reordered.show(10)

+-------+----------+--------+------+------------+---------------+------------------+--------------+--------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+--------------------+-------------------+-------------------+-------------+----------+------------+------------------+------------------+-------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+-------------------+--------------------+--------------------+--------------------+----------+--------------------+-------------------+---------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up

In [55]:
submission_sdf = spark.read.csv('data/instacart_market/sample_submission.csv', header=True, inferSchema=True)
print(submission_sdf.count())
print(orders_sdf.filter(F.col('eval_set') == 'test').count())
submission_sdf.show(truncate=False)

75000
75000
+--------+-----------+
|order_id|products   |
+--------+-----------+
|17      |39276 29259|
|34      |39276 29259|
|137     |39276 29259|
|182     |39276 29259|
|257     |39276 29259|
|313     |39276 29259|
|353     |39276 29259|
|386     |39276 29259|
|414     |39276 29259|
|418     |39276 29259|
|437     |39276 29259|
|452     |39276 29259|
|474     |39276 29259|
|492     |39276 29259|
|497     |39276 29259|
|513     |39276 29259|
|517     |39276 29259|
|604     |39276 29259|
|680     |39276 29259|
|758     |39276 29259|
+--------+-----------+
only showing top 20 rows



In [56]:
test_orders_sdf = orders_sdf.filter(F.col('eval_set') == 'test')
test_orders_sdf.orderBy('order_id').show()

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
|      17|  36855|    test|           5|        6|               15|                   1.0|
|      34|  35220|    test|          20|        3|               11|                   8.0|
|     137| 187107|    test|           9|        2|               19|                  30.0|
|     182| 115892|    test|          28|        0|               11|                   8.0|
|     257|  35581|    test|           9|        6|               23|                   5.0|
|     313| 113359|    test|          31|        6|               22|                   7.0|
|     353| 173814|    test|           4|        4|               13|                  30.0|
|     386|  55492|    test|           8|        0|               15|            

In [57]:
submission_sdf.createOrReplaceTempView('submission')

In [58]:
spark.catalog.listTables()

[Table(name='data_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order_priors_prods', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order_trains_prods', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='prd_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='test_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='train_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='up_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='user_mart', catalog='spark_catalog', namespace=

In [59]:
query = '''
select count(*) 
from test_data t
left outer join submission s
on t.order_id = s.order_id
where s.order_id is null
'''
spark.sql(query).show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [60]:
predictions.show(10)

+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------------------+---------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|

In [61]:
predictions_grp = predictions.groupBy('order_id').agg(
    F.count('*').alias('total_cnt_by_order_id'),
    F.sum(F.col('reordered')).alias('reordered_cnt')
)
print(predictions_grp.count(), predictions_grp.filter(F.col('reordered_cnt') == 0).count())

75000 8666


In [62]:
predictions_grp.filter(F.col('reordered_cnt') == 0).orderBy('order_id').show()

+--------+---------------------+-------------+
|order_id|total_cnt_by_order_id|reordered_cnt|
+--------+---------------------+-------------+
|     353|                   12|            0|
|     474|                   21|            0|
|     513|                   16|            0|
|    1195|                   16|            0|
|    1564|                   20|            0|
|    1789|                   17|            0|
|    2297|                   10|            0|
|    2887|                  111|            0|
|    3519|                   15|            0|
|    5216|                   23|            0|
|    6138|                   85|            0|
|    6222|                   27|            0|
|    7125|                   43|            0|
|    7373|                   60|            0|
|    7671|                   53|            0|
|    7736|                   20|            0|
|    8030|                   21|            0|
|    8450|                   51|            0|
|    9318|   

In [63]:
predictions.filter(F.col('reordered') == 1).groupBy('order_id').count().orderBy('order_id').show(10)

+--------+-----+
|order_id|count|
+--------+-----+
|      17|    1|
|      34|    6|
|     137|    7|
|     182|    5|
|     257|    7|
|     313|    6|
|     386|   13|
|     414|    3|
|     418|    5|
|     437|    2|
+--------+-----+
only showing top 10 rows



In [64]:
predictions.filter(F.col('reordered') == 1).groupBy('order_id').agg(F.collect_list('product_id')).orderBy('order_id').show(10, truncate=False)

+--------+------------------------------------------------------------------------------------------+
|order_id|collect_list(product_id)                                                                  |
+--------+------------------------------------------------------------------------------------------+
|17      |[13107]                                                                                   |
|34      |[39475, 47766, 21137, 43504, 47792, 16083]                                                |
|137     |[23794, 24852, 41787, 38689, 25890, 5134, 2326]                                           |
|182     |[47672, 39275, 33000, 5479, 9337]                                                         |
|257     |[27966, 29837, 4605, 24852, 49235, 27104, 30233]                                          |
|313     |[12779, 13198, 45007, 21903, 46906, 28535]                                                |
|386     |[21479, 42265, 39180, 4920, 30450, 15872, 22124, 47766, 37935, 24852, 45

In [65]:
def get_product_ids_str(product_id_group):
    product_ids_str = ''
    for product_id in product_id_group:
        product_ids_str += f' {product_id}'
    return product_ids_str
udf_get_product_ids_str = F.udf(get_product_ids_str)

In [66]:
submission_01 = predictions.filter(F.col('reordered') == 1).groupBy('order_id').agg(
    udf_get_product_ids_str(F.collect_list('product_id')).alias('products')
)
submission_01.show(10)

+--------+--------------------+
|order_id|            products|
+--------+--------------------+
|      34| 39475 47766 2113...|
|     137| 23794 24852 4178...|
|     182| 47672 39275 3300...|
|     386| 21479 42265 3918...|
|     452| 21137 36735 6046...|
|     497| 36316 39947 3196...|
|     604|   24852 12099 16797|
|     758|               19660|
|     887|         24852 25647|
|    1304|   42265 24852 22035|
+--------+--------------------+
only showing top 10 rows



In [67]:
submission_02 = predictions_grp.filter(F.col('reordered_cnt') == 0).withColumn(
    'products',
    F.lit('None')
).select('order_id', 'products')
submission_02.show(10)

+--------+--------+
|order_id|products|
+--------+--------+
|  488000|    None|
| 1107801|    None|
|  471276|    None|
|  200687|    None|
| 2726972|    None|
|  144907|    None|
|  743061|    None|
| 1552436|    None|
| 1833275|    None|
| 1274216|    None|
+--------+--------+
only showing top 10 rows



In [68]:
submission = submission_01.union(submission_02)
print('submission count:', submission.count())
submission = submission.orderBy('order_id')
submission.show(submission.count())

submission count: 75000


+--------+--------------------+
|order_id|            products|
+--------+--------------------+
|      17|               13107|
|      34| 39475 47766 2113...|
|     137| 23794 24852 4178...|
|     182| 47672 39275 3300...|
|     257| 27966 29837 4605...|
|     313| 12779 13198 4500...|
|     353|                None|
|     386| 21479 42265 3918...|
|     414|   20564 20392 27845|
|     418| 30489 38694 4195...|
|     437|          27966 4589|
|     452| 21137 36735 6046...|
|     474|                None|
|     492| 21137 49683 4463...|
|     497| 36316 39947 3196...|
|     513|                None|
|     517| 24852 37646 2190...|
|     604|   24852 12099 16797|
|     680| 47977 7743 14836...|
|     758|               19660|
|     759| 47766 24852 1465...|
|     887|         24852 25647|
|    1161| 26209 22935 4414...|
|    1195|                None|
|    1304|   42265 24852 22035|
|    1408| 47912 47626 4667...|
|    1513|    26250 29373 5385|
|    1564|                None|
|    172

In [75]:
submission.coalesce(1).write.csv('data/instacart_market/submission.csv', header=True)